In [1]:
#For a target protein, minimize and add in FAD and
#Inputs: 
    #model/models/{model}/ranked_0.pdb
    #cofactor/{model}/{model}_cdocker.pdb
    #cofactor/{model}/fad_in.pdb (from tropb_cdocker.pdb)
#Outputs:
    #cofactor/model_align/{model}.pdb
    #cofactor/pdb_with_fad/{model}_fad.pdb
import os
import sys

#Set CHARMM lib dir for pyCHARMM
os.environ['CHARMM_LIB_DIR'] = "/home/azamh/charmm/c48a/pycharmm_build/install/lib/"

# These are a subset of the pycharmm modules that were installed when
# pycharmm was installed in your python environment
import pycharmm
import pycharmm.generate as gen
import pycharmm.ic as ic
import pycharmm.coor as coor
import pycharmm.energy as energy
import pycharmm.dynamics as dyn
import pycharmm.nbonds as nbonds
import pycharmm.minimize as minimize
import pycharmm.crystal as crystal
import pycharmm.image as image
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.cons_harm as cons_harm
import pycharmm.cons_fix as cons_fix
import pycharmm.select as select
import pycharmm.shake as shake
import pycharmm.settings as settings
from pycharmm.select_atoms import SelectAtoms
from pycharmm.lingo import charmm_script
from pycharmm.lib import charmm as libcharmm

In [2]:
#Arguments
#Protein to add FAD cofactor to
model = 'tropb'
#Contains tropb_cdocker.pdb, a QM/MM refined structure that has 3mo docked with tropb using CDOCKER and structures of fad IN
tropbdir = '../tropb'
#Contains rtf and parameter files for CHARMM
toppardir = '../../toppar'
#Directory that contains AlphaFold structures
modeldir = '../../model/models'
#Directory containing alphafold models after being aligned with tropb_cdocker.pdb
model_align_dir = '../model_align'
#Directory with alphafold models and FAD, with cofactor taken from aligned tropb_cdocker.pdb
pdb_with_fad_dir = '../pdb_with_fad'

#Structure to generate
pdb_with_fad = os.path.join(pdb_with_fad_dir, f'{model}_fad.pdb')
pdb_with_fad

'../pdb_with_fad/tropb_fad.pdb'

In [3]:
#Step 1: Align model with QM/MM refined structure of tropb docked with 3mo (could align with crystal structure as well)

#Generates model.psf and model.pdb in model_align directory
#Run TM align: superpose model with tropb structure
os.system(f'./TMalign {modeldir}/{model}/ranked_0.pdb {tropbdir}/tropb_cdocker.pdb -o {model_align_dir}/{model}_sup')

#Run convpdb on superposed model so that pdb is in charmm readible format
os.system(f'convpdb.pl -out charmm22 -nohetero -segnames -nsel A: {model_align_dir}/{model}_sup.pdb > {model_align_dir}/{model.lower()}_conv.pdb')
model=model.lower() 


 *********************************************************************
 * TM-align (Version 20190822): protein structure alignment          *
 * References: Y Zhang, J Skolnick. Nucl Acids Res 33, 2302-9 (2005) *
 * Please email comments and suggestions to yangzhanglab@umich.edu   *
 *********************************************************************

Name of Chain_1: ranked_0.pdb (to be superimposed onto Chain_2)
Name of Chain_2: tropb_cdocker.pdb
Length of Chain_1: 447 residues
Length of Chain_2: 437 residues

Aligned length= 432, RMSD=   1.85, Seq_ID=n_identical/n_aligned= 0.958
TM-score= 0.92232 (if normalized by length of Chain_1, i.e., LN=447, d0=7.57)
TM-score= 0.94270 (if normalized by length of Chain_2, i.e., LN=437, d0=7.50)
(You should use TM-score normalized by length of the reference structure)

(":" denotes residue pairs of d <  5.0 Angstrom, "." denotes other aligned residues)
MPGSLIDTRQQPLSVGIVGGGIIGVILAAGLVRRGIDVKVFEQARGFREIGAGMAFTANAVRCMEMLDPAIVWALRSSGAVPISIGD-----

In [4]:
#Minimize model in vacuum to relax protein 

#Read in topology and parameter files
settings.set_bomb_level(-1)
read.rtf(os.path.join(toppardir, 'top_all36_prot.rtf'))
read.rtf(os.path.join(toppardir, 'top_all36_cgenff.rtf'), append = True)
read.prm(os.path.join(toppardir, 'par_all36m_prot.prm'), flex = True)
read.prm(os.path.join(toppardir, 'par_all36_cgenff.prm'), flex = True, append = True)
charmm_script(f'stream {os.path.join(toppardir, "st2_fadh.str")}') #Parameters for FAD cofactor
settings.set_bomb_level(0)

  
 CHARMM>     read rtf card -
 CHARMM>     name ../../toppar/top_all36_prot.rtf
 VOPEN> Attempting to open::../../toppar/top_all36_prot.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> MAY 2011 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read rtf card -
 CHARMM>     name ../../toppar/top_all36_cgenff.rtf -
 CHARMM>     append
 VOPEN> Attempting to open::../../toppar/top_all36_cgenff.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *  --------------------------------------------------------------------------  *
 TITLE> *          CGENFF: TOPOLOGY FOR THE CHARMM GENERAL FORCE FIELD 

-1

In [5]:
#Read in protein sequence and generate PSF
read.sequence_pdb(os.path.join(model_align_dir, f'{model}_conv.pdb'))
gen.new_segment(seg_name='PROA', first_patch='ACE', last_patch='CT3', setup_ic=True)
read.pdb(os.path.join(model_align_dir, f'{model}_conv.pdb'),resid=True)
ic.prm_fill(replace_all=False)
ic.build()
write.psf_card(os.path.join(model_align_dir, f'{model}.psf'))

  
 CHARMM>     read sequence pdb -
 CHARMM>     name ../model_align/tropb_conv.pdb
 VOPEN> Attempting to open::../model_align/tropb_conv.pdb::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --   447 RESIDUES
          MET PRO GLY SER LEU ILE ASP THR ARG GLN GLN PRO LEU SER VAL GLY ILE VAL GLY GLY 
          GLY ILE ILE GLY VAL ILE LEU ALA ALA GLY LEU VAL ARG ARG GLY ILE ASP VAL LYS VAL 
          PHE GLU GLN ALA ARG GLY PHE ARG GLU ILE GLY ALA GLY MET ALA PHE THR ALA ASN ALA 
          VAL ARG CYS MET GLU MET LEU ASP PRO ALA ILE VAL TRP ALA LEU ARG SER SER GLY ALA 
          VAL PRO ILE SER ILE GLY ASP HSD GLN ALA GLU ALA ARG ASP TYR LEU ARG TRP VAL ASP 
          GLY TYR HSE GLU SER SER LYS ARG LEU TYR GLN LEU ASP ALA GLY ILE ARG GLY PHE GLU 
          ALA CYS ARG ARG ASP GLN PHE LEU GLU ALA LEU VAL LYS VAL LEU PRO GLU GLY ILE VAL 
          GLU CYS GLN LYS ARG LEU GLN LYS ILE HSE GLU LYS ASN GLU THR GLU LYS VAL THR LEU 
          GLU P

In [6]:
#Setup nonbonds
my_nbonds = pycharmm.NonBondedScript(
    cutnb=10.0, ctonnb=8.0, ctofnb=7.0,
    eps=1.0,
    cdie=True,
    atom=True, vatom=True,
    fswitch=True, vfswitch=True)
# Implement these non-bonded parameters by "running" them.
my_nbonds.run()

  
 CHARMM>     nbonds cutnb 10.0 -
 CHARMM>     ctonnb 8.0 -
 CHARMM>     ctofnb 7.0 -
 CHARMM>     eps 1.0 -
 CHARMM>     cdie -
 CHARMM>     atom -
 CHARMM>     vatom -
 CHARMM>     fswitch -
 CHARMM>     vfswitch

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  FSWItch  VATOm    VFSWIt  
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 10.000 CTEXNB =999.000 CTONNB =  8.000 CTOFNB =  7.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  1.000
 NBXMOD =      5
 There are        0 atom  pairs and        0 atom  exclusions.
 There are        0 group pairs and        0 group exclusions.
 GTNBCT> CUTNB,CTOFNB,CTONNB=      10.0       7.0       8.0

      ***** LEVEL  1 WARNING FROM <GTNBCT> *****
      ***** CUTNB,CTOFNB,CTONNB are not in correct order.
      ******************************************
      BOMLEV (  0) IS NOT REACHED. WRNLEV IS  5

 <MAKINB> with mode   5 found  19718 exclusions and  18198 interactions(1-4)
 <MAKGRP>

In [7]:
#Impose restraints on heavy atoms
cons_harm.setup_absolute(selection=~pycharmm.SelectAtoms(hydrogens=True),force_constant=50)
#minimize with steepest descent 
minimize.run_sd(nstep=1000, tolenr=1e-3, tolgrd=1e-4)
cons_harm.turn_off()

 CSTRAN: Harmonic Restraints
          ABSOlute type as set number  1.  Number of selected atoms:   3516
          Reference coordinates set to main coordinates.
          Mass weighting will NOT be used for new restraints.
          The force constant of       0.00000 will be used.
          An exponent of  2 will be used.
          The XYZ scale factors are:       1.00000       1.00000       1.00000
          A total of      0 atoms are restrained.

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  FSWItch  VATOm    VFSWIt  
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 10.000 CTEXNB =999.000 CTONNB =  8.000 CTOFNB =  7.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  1.000
 NBXMOD =      5
 There are  1006987 atom  pairs and    37916 atom  exclusions.
 There are        0 group pairs and     5960 group exclusions.
 GTNBCT> CUTNB,CTOFNB,CTONNB=      10.0       7.0       8.0

      ***** LEVEL  1 WARNING FROM <GTNBCT> *****
    

1

In [8]:
#Save minimized pdb
write.coor_pdb(os.path.join(model_align_dir, f'{model}_minimized.pdb'))

  
 CHARMM>     write name ../model_align/tropb_minimized.pdb -
 CHARMM>     coor pdb
 VOPEN> Attempting to open::../model_align/tropb_minimized.pdb::
 RDTITL>  
 RDTITL> No title read.
  Write CHARMM-pdb format
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  


In [9]:
#Step 2: Place in FAD cofactor from aligned tropb structure (fad_in.pdb/fad_in.psf)
#Generates model_fad.pdb and model_fad.psf in pdb_with_fad directory 

#Generate FAD
read.sequence_pdb(os.path.join(tropbdir, 'fad_in.pdb'))
gen.new_segment(seg_name='FADH', setup_ic=True)
read.pdb(os.path.join(tropbdir, 'fad_in.pdb'), resid=True)
charmm_script('auto angle dihe')
write.psf_card(os.path.join(pdb_with_fad_dir, f'{model}_fad.psf'))

  
 CHARMM>     read sequence pdb -
 CHARMM>     name ../tropb/fad_in.pdb
 VOPEN> Attempting to open::../tropb/fad_in.pdb::
 MAINIO> Sequence information being read from unit  91.
 TITLE>    DATE:     4/12/21     23: 4:36      CREATED BY USER: AZAMH
 TITLE>  *

          RESIDUE SEQUENCE --     1 RESIDUES
          FAD     
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
 NO PATCHING WILL BE DONE ON THE FIRST RESIDUE
 NO PATCHING WILL BE DONE ON THE LAST  RESIDUE
 AUTGEN: Autogenerating specified angles and dihedrals.
 GENPSF> Segment   2 has been generated. Its identifier is FADH.
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        2   Number of residues   =      448
         Number of atoms         =     7040   Number of groups     =     2042
         Number of bonds         =     7130   Number of angles     =    12832
         Number of dihedrals     =    18767   N

In [10]:
#Select backbone atoms and flexible side chains around FAD
protein_sel = SelectAtoms(seg_id="PROA") & ~SelectAtoms(hydrogens=True)
fad_sel = SelectAtoms(seg_id="FADH")
all_sel = SelectAtoms(select_all=True)

backbone_sel = protein_sel & \
    (SelectAtoms(atom_type="CA") | \
     SelectAtoms(atom_type="N") | \
     SelectAtoms(atom_type="C") | \
     SelectAtoms(atom_type="O")
    )
flexres_sel = (pycharmm.SelectAtoms(seg_id="PROA") & \
              pycharmm.SelectAtoms(seg_id="FADH").around(5)).whole_residues()
flexres_sc_sel = flexres_sel & (~backbone_sel)

print('all atoms:', all_sel.get_n_selected())
print('protein atoms:', protein_sel.get_n_selected())
print('fad atoms:', fad_sel.get_n_selected())
print('backbone atoms', backbone_sel.get_n_selected())
print('flexible residues around FAD atoms:', flexres_sel.get_n_selected())
print('flexible residues sidechains around FAD atoms:', flexres_sc_sel.get_n_selected())

all atoms: 7040
protein atoms: 3516
fad atoms: 84
backbone atoms 1788
flexible residues around FAD atoms: 635
flexible residues sidechains around FAD atoms: 471


In [11]:
#restrain backbone atoms first letting sidechains and fad move
cons_harm.setup_absolute(selection=backbone_sel,force_constant=50)
minimize.run_sd(nstep=200, tolenr=1e-3, tolgrd=1e-4)
cons_harm.turn_off()

 CSTRAN: Harmonic Restraints
          ABSOlute type as set number  1.  Number of selected atoms:   1788
          Reference coordinates set to main coordinates.
          Mass weighting will NOT be used for new restraints.
          The force constant of       0.00000 will be used.
          An exponent of  2 will be used.
          The XYZ scale factors are:       1.00000       1.00000       1.00000
          A total of      0 atoms are restrained.

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  FSWItch  VATOm    VFSWIt  
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 10.000 CTEXNB =999.000 CTONNB =  8.000 CTOFNB =  7.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  1.000
 NBXMOD =      5
 There are        0 atom  pairs and        0 atom  exclusions.
 There are        0 group pairs and        0 group exclusions.
 GTNBCT> CUTNB,CTOFNB,CTONNB=      10.0       7.0       8.0

      ***** LEVEL  1 WARNING FROM <GTNBCT> *****
    

1

In [12]:
#restrain all atoms except nearby side chains and fad
cons_harm.setup_absolute(selection=((~flexres_sc_sel) & protein_sel),force_constant=50)
minimize.run_sd(nstep=200, tolenr=1e-3, tolgrd=1e-4)
cons_harm.turn_off()

 CSTRAN: Harmonic Restraints
          ABSOlute type as set number  1.  Number of selected atoms:   3360
          Reference coordinates set to main coordinates.
          Mass weighting will NOT be used for new restraints.
          The force constant of       0.00000 will be used.
          An exponent of  2 will be used.
          The XYZ scale factors are:       1.00000       1.00000       1.00000
          A total of      0 atoms are restrained.

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  FSWItch  VATOm    VFSWIt  
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 10.000 CTEXNB =999.000 CTONNB =  8.000 CTOFNB =  7.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  1.000
 NBXMOD =      5
 There are  1032448 atom  pairs and    38361 atom  exclusions.
 There are        0 group pairs and     5960 group exclusions.
 GTNBCT> CUTNB,CTOFNB,CTONNB=      10.0       7.0       8.0

      ***** LEVEL  1 WARNING FROM <GTNBCT> *****
    

1

In [13]:
#restrain everything except fad
cons_harm.setup_absolute(selection=protein_sel,force_constant=50)
minimize.run_sd(nstep=200, tolenr=1e-3, tolgrd=1e-4)
minimize.run_abnr(nstep=1000, tolenr=1e-3, tolgrd=1e-4)
cons_harm.turn_off()

 CSTRAN: Harmonic Restraints
          ABSOlute type as set number  1.  Number of selected atoms:   3516
          Reference coordinates set to main coordinates.
          Mass weighting will NOT be used for new restraints.
          The force constant of       0.00000 will be used.
          An exponent of  2 will be used.
          The XYZ scale factors are:       1.00000       1.00000       1.00000
          A total of      0 atoms are restrained.

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  FSWItch  VATOm    VFSWIt  
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 10.000 CTEXNB =999.000 CTONNB =  8.000 CTOFNB =  7.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  1.000
 NBXMOD =      5
 There are  1032494 atom  pairs and    38361 atom  exclusions.
 There are        0 group pairs and     5960 group exclusions.
 GTNBCT> CUTNB,CTOFNB,CTONNB=      10.0       7.0       8.0

      ***** LEVEL  1 WARNING FROM <GTNBCT> *****
    

1

In [14]:
#print energy
energy.show()


 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  FSWItch  VATOm    VFSWIt  
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 10.000 CTEXNB =999.000 CTONNB =  8.000 CTOFNB =  7.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  1.000
 NBXMOD =      5
 There are  1036541 atom  pairs and    38361 atom  exclusions.
 There are        0 group pairs and     5960 group exclusions.
 GTNBCT> CUTNB,CTOFNB,CTONNB=      10.0       7.0       8.0

      ***** LEVEL  1 WARNING FROM <GTNBCT> *****
      ***** CUTNB,CTOFNB,CTONNB are not in correct order.
      ******************************************
      BOMLEV (  0) IS NOT REACHED. WRNLEV IS  5

 Generating nonbond list with Exclusion mode = 5
 == PRIMARY == SPACE FOR  1092668 ATOM PAIRS AND        0 GROUP PAIRS
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       19.76 Angstroms and starts with atom:    6957
   Please check group boundary definitions.

 General atom nonbon

In [15]:
#Save pdb
write.coor_pdb(os.path.join(pdb_with_fad_dir, f'{model}_fad.pdb'))

  
 CHARMM>     write name ../pdb_with_fad/tropb_fad.pdb -
 CHARMM>     coor pdb
 VOPEN> Attempting to open::../pdb_with_fad/tropb_fad.pdb::
 RDTITL>  
 RDTITL> No title read.
  Write CHARMM-pdb format
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
